In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
def find_zip_by_address(address):
    address = '+'.join(address.split(' '))
    url = "https://www.zip-codes.com/search.asp?fld-address=%s&fld-address2=&fld-city=New+York&fld-state=NY&fld-zip=&srch-type=address&selectTab=1&Submit=Find+ZIP+Code+of+this+U.S.+Address"%address
    response=requests.get(url)
    page_data_soup=BeautifulSoup(response.content,'lxml')
    if response.status_code==200:
        results=page_data_soup.find('div',style='font-size: 22px; text-align:center; padding-top:20px; padding-top:30px;')
        if results is None:
            return 'None'
        else:
            zip_code=results.find('span')
            return zip_code.text[0:5]
    else:
        print ("Something wrong with this address, and we can't return the Zip")

def extract_from_rent_com(address,bedroom,min_price,max_price):
    data=pd.DataFrame(columns = ["address","price/$","bdrms","baths","link","latitude","longitude"])
    zipcode=find_zip_by_address(address)
    url="http://www.rent.com/zip-%s-houses/%s-bedroom_min-price-%s_max-price-%s"%(zipcode,bedroom,min_price,max_price)
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    #to get the total listing amount
    total_listings_count=page_data_soup.find('span',class_="total-listings-count").text 
    page_count = 1
    listings_count = 1
    #loop through each page until we go through all the apts
    while listings_count<=int(total_listings_count):
        url = "http://www.rent.com/zip-%s-houses/%s-bedroom_min-price-%s_max-price-%s?page=%s"%(zipcode,bedroom,min_price,max_price,page_count)
        response = requests.get(url)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        ori_url="http://www.rent.com"
        apt_each_page=page_data_soup.find_all('div',class_="prop-info result")
        for j in range(len(apt_each_page)):
            single_result = page_data_soup.find_all('div',class_="prop-info result")[j]
            add_where = single_result.find('div',class_="prop-title clearfix")
            addr_link = ori_url+add_where.find('a').get("href")
            addr = add_where.find('strong').text
            price_where = single_result.find('p',class_="prop-rent")       
            price = price_where.find('span').text.strip()
            room_where = single_result.find('div',class_="prop-unit-info bullet-separator")
            bed = room_where.find('span',class_="prop-beds bullet-separator").text.strip()[0]
            bath = room_where.find('span',class_="prop-baths bullet-separator").text.strip()[0]
            #write the info of each apts into the row of dataframe
            coordinate = get_lat_lng_json(addr,country='USA')
            try:
                lat = coordinate[0]
                lng = coordinate[1]
                data.loc[data.shape[0]] = [addr,price,bed,bath,addr_link,lat,lng]
                sql = "INSERT INTO da_web_scraping VALUES (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\');"%(addr,price,bed,bath,addr_link,lat,lng)
                cursor.execute(sql)
            #to help finish the while loop
            except:
                pass
            
            listings_count+= 1
        #go to next page
        page_count+=1
    return data

def extract_from_zillow_com(address,bedroom,min_price,max_price):
    data = pd.DataFrame(columns = ["address","price/$","bdrms","baths","link","latitude","longitude"])
    zipcode = find_zip_by_address(address)
    url = "http://www.zillow.com/homes/for_rent/Manhattan-New-York-NY-%s/%s-_beds/%s-%s_mp/"%(zipcode,bedroom,min_price,max_price)
    ori_url = "http://www.zillow.com"
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    #to get the total_page_count
    try:
        total_page_count = int(page_data_soup.find("ol",class_="zsg-pagination").text.strip()[-5])
    except:
        total_page_count = 1
    page_count = 1
    #loop through all the pages
    while page_count<=int(total_page_count):
        url = "http://www.zillow.com/homes/for_rent/Manhattan-New-York-NY-%s/%s-_beds/%s-%s_mp/%s_p/"%(zipcode,bedroom,min_price,max_price,page_count)
        response = requests.get(url)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        #get all the apts on one page
        apt_each_page = page_data_soup.find_all("div",class_="zsg-photo-card-content zsg-aspect-ratio-content")
        #loop through each apt on one page
        for j in range(len(apt_each_page)):
            single_result = page_data_soup.find_all('div',class_="zsg-photo-card-content zsg-aspect-ratio-content")[j]
            addr = single_result.find('span',class_="zsg-photo-card-address").text
            addr_link = ori_url+single_result.find('a').get("href")
            try:
                ori_price = single_result.find('span',class_="zsg-photo-card-price").text
                price = "".join(ori_price.split(','))
                #'+'.join(address.split(' '))
                bed = single_result.find('span',class_="zsg-photo-card-info").text.strip()[0]
                try:
                    bath_where = single_result.find('span',class_="zsg-photo-card-info").text.strip()
                    if bath_where[0] == str(1):
                        bath = bath_where[7]
                    else:
                        bath = bath_where[8]
                except:
                    bath = 'Nan'
            except:
                for i in single_result.find_all('span',class_="zsg-photo-card-unit"):
                    ori_price = i.text.strip()[1:]
                    price = "".join(ori_price.split(','))
                    bed = i.text.strip()[0]
                    bath = 'Nan'
            #write the info of each apt into the dataframe as a row
            coordinate = get_lat_lng_json(addr,country='USA')
            try:
                lat = get_lat_ltng_json(addr,country='USA')[0]
                lng = get_lat_lng_json(addr,country='USA')[1]
                data.loc[data.shape[0]] = [addr,price,bed,bath,addr_link,lat,lng]
                sql = "INSERT INTO da_web_scraping VALUES (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\');"%(addr,price,bed,bath,addr_link,lat,lng)
                cursor.execute(sql)
            except:
                pass
        page_count+=1
    return data

def extract_from_realtor_com(address,bedroom,min_price,max_price):
    data = pd.DataFrame(columns = ["address","price/$","bdrms","baths","link","latitude","longitude"])
    zipcode = find_zip_by_address(address)
    url="http://www.realtor.com/apartments/%s/beds-%s/price-%s-%s"%(zipcode,bedroom,min_price,max_price)
    ori_url="http://www.realtor.com"
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    #to get the total_page_count
    try:
        total_page_count = int(page_data_soup.find_all('span',class_="page")[-1].text)
    except:
        total_page_count = 1

    page_count = 1
    #loop through all the pages
    while page_count<=int(total_page_count):
        url="http://www.realtor.com/apartments/%s/beds-%s/price-%s-%s/pg-%s"%(zipcode,bedroom,min_price,max_price,page_count)
        response = requests.get(url)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        #get all the apts on one page
        apt_each_page = page_data_soup.find_all("div",class_="aspect-content")
        #loop through each apt on one page
        for j in range(len(apt_each_page)):
            single_result = page_data_soup.find_all('div',class_="aspect-content")[j]
            addr = single_result.find('span',class_="listing-street-address").text
            addr_link = ori_url+single_result.find('a').get("href")
            try:      
                ori_price=single_result.find('li',class_="srp-item-price").text.strip()
                price = "".join(ori_price.split(','))
            except:
                price="Nan"
            see_if_mutli=len(single_result.find('li', {"data-label":"property-meta-beds"}).text)
            bed=single_result.find('li', {"data-label":"property-meta-beds"}).text.strip()[0]
            bath=single_result.find('li', {"data-label":"property-meta-baths"}).text.strip()[0]            
            if see_if_mutli==4:
                coordinate = get_lat_lng_json(addr,country='USA')
                try:
                    lat = get_lat_lng_json(addr,country='USA')[0]
                    lng = get_lat_lng_json(addr,country='USA')[1]
                    data.loc[data.shape[0]]=[addr,price,bed,bath,addr_link,lat,lng]
                    sql = "INSERT INTO da_web_scraping VALUES (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\');"%(addr,price,bed,bath,addr_link,lat,lng)
                    cursor.execute(sql)
                except:
                    pass
        page_count+=1
    return data

def extract_from_trulia(address,bed,min_price,max_price):
    zip_code = find_zip_by_address(address)
    url = 'https://www.trulia.com/for_rent/{}_zip/{}p_beds/{}-{}_price'.format(zip_code,bed,min_price,max_price)
    response = requests.get(url)
    if response.status_code == 200:
        page = BeautifulSoup(response.content,'html.parser')
        p=[]
    for i in range(100):
        test = page.find('div',{"class":"col cols16 mts txtC srpPagination_list"})
        if test is not None:
            for all in test.find_all('a'):
                #print(all.text)
                p.append(all.text)
            break
        else:
            continue
    try:
        max_page = int(p[-1])   
    except:
        max_page = 1

    l= list()
    for page in range(max_page):
        url = 'https://www.trulia.com/for_rent/{}_zip/{}p_beds/{}-{}_price/{}_p'.format(zip_code,bed,min_price,max_price,page+1)
        response = requests.get(url)
        if response.status_code == 200:
            page = BeautifulSoup(response.content,'lxml')
            for tag in page.find_all('li',{'class':"hover propertyCard property-data-elem clickable"}):
                #address
                address = tag.find('span',{'itemprop':"streetAddress"}).get_text()
                #price
                ori_price = tag.find('span',{'class':"typeEmphasize"}).get_text().strip()
                price = "".join(ori_price.split(','))
                #bathroom and bedroom
                b=[]
                for subtag in tag.find('div',{'class':'col cols3'}).find_all('div'):
                    b.append(subtag.get_text().strip()[0])
                #link
                link = tag.find('a',{'class':'primaryLink pdpLink activeLink'}).get('href')
                link = 'https://www.trulia.com' + link
                coordinate = get_lat_lng_json(address,country='USA')
                try:
                    lat = get_lat_lng_json(address,country='USA')[0]
                    lng = get_lat_lng_json(address,country='USA')[1]                
                    l.append((address,price,b[0],b[1],link,lat,lng))
                    sql = "INSERT INTO da_web_scraping VALUES (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\');"%(address,price,b[0],b[1],link,lat,lng)
                    cursor.execute(sql)
                except:
                    pass
        else:
            print('We cannot find the website')
    data=pd.DataFrame(l,columns=["address","price/$","bdrms","baths","link","latitude","longitude"])
    return data

def extract_from_showmetherent(address,bed,min_price,max_price):
    zip_code = find_zip_by_address(address)
    url = url = "https://www.showmetherent.com/listings/{}/beds:{}/rent:{}-{}".format(zip_code,bed,min_price,max_price)
    response = requests.get(url)
    page = BeautifulSoup(response.content,'lxml')
    p = []
    for page_count in page.find('div',{'class':'page_bar'}).find_all('a'):
        p.append(page_count.get_text())
    max_page = p[-2]
    
    l= list()
    for page in range(int(max_page)):
        url = 'https://www.showmetherent.com/listings/{}/beds:{}/rent:{}-{}/start:{}'.format(zip_code,bed,min_price,max_price,page*20)
        response = requests.get(url)
        if response.status_code == 200:
            page = BeautifulSoup(response.content,'lxml')
        
            for tag in page.find_all('a',{'class':'listing-tile-mobile'}):
                #link
                link = tag.get('href')
                link = 'https://www.showmetherent.com'+link
                response_house = requests.get(link)
                page_house = BeautifulSoup(response_house.content,'lxml')
                #address
                address = page_house.find('h1').get_text()
                #price
                ori_price = page_house.find('span',{'class':"property-unit-rent"}).get_text().strip()
                price = "".join(ori_price.split(','))
                #bathroom and bedroom
                bedroom = " ".join(page_house.find('p',{'class':"property-unit-details"}).get_text().split()[0:2]).strip()[0]
                bathroom = " ".join(page_house.find('p',{'class':"property-unit-details"}).get_text().split()[2:4]).strip()[0]
                coordinate = get_lat_lng_json(address,country='USA')
                try:
                    lat = get_lat_lng_json(address,country='USA')[0]
                    lng = get_lat_lng_json(address,country='USA')[1]                
                    l.append((address,price,bedroom,bathroom,link,lat,lng))
                    sql = "INSERT INTO da_web_scraping VALUES (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\');"%(address,price,bedroom,bathroom,link,lat,lng)
                    cursor.execute(sql)
                except:
                    pass
        else:
            print('We cannot find the website')
    #data=pd.DataFrame(l,columns=["address","price/$","bdrms","baths","link"])
    data=pd.DataFrame(l,columns=["address","price/$","bdrms","baths","link","latitude","longitude"])
    return data

In [2]:
def get_lat_lng_json(address,country='ALL'):
    address = '_'.join(address.split(' '))
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s" % (address)
    import requests
    response = requests.get(url)
    i=0
    if response.status_code == 200:
        coordinate = []
        if country=='ALL':
            for x in response.json()['results']:
                coordinate.append(x['geometry']['location']['lat'])
                coordinate.append(x['geometry']['location']['lng'])
        else:
            for x in response.json()['results']:
                if x['formatted_address'].split()[-1].lower()==country.lower():
                    i=i+1
                    coordinate.append(x['geometry']['location']['lat'])
                    coordinate.append(x['geometry']['location']['lng'])
                else:
                    continue
            if i==0:
                coordinate.append('Nan')
        return coordinate
    else:
        return None

In [4]:
db = pymysql.connect("localhost","root","123456","DA_project")
cursor = db.cursor()
cursor.execute("use DA_project;")

d1=extract_from_rent_com('256w 108th st',1,1200,1250)
d2=extract_from_zillow_com('256w 108th st',1,1200,1250)
d3=extract_from_realtor_com('256w 108th st',1,1200,1250)
d4=extract_from_showmetherent('256w 108th st',1,1200,1250)
#d5=extract_from_trulia('256w 108th st',1,1200,2500)
data_combine=pd.concat([d1,d2,d3,d4],ignore_index=True)

data_combine.to_excel('test_coordinate.xlsx',sheet_name='Sheet 1')


In [5]:
cursor.execute("SELECT DISTINCT * FROM DA_project.da_web_scraping;")

67

In [6]:
for thing in cursor.fetchall():
    print(thing)


('Anderson Ave and Clark Terrace', '$1250', '1', '1', 'http://www.rent.com/new-jersey/cliffside-park-houses/anderson-ave-and-clark-terrace-4-lv25537765', '40.8198994', '-73.9921002')
('91st St and Bergenwood Ave', '$1250', '1', '1', 'http://www.rent.com/new-jersey/north-bergen-houses/91st-st-and-bergenwood-ave-4-lv25537767', '40.8111757', '-74.0058822')
('1115 Jerome Avenue', '$1250', '1', '1', 'http://www.rent.com/new-york/new-york-houses/1115-jerome-avenue-4-lv25990902', '40.8348119', '-73.92453259999999')
('1265 Walton Avenue', '$1250', '1', '1', 'http://www.rent.com/new-york/new-york-houses/1265-walton-avenue-4-lv25964631', '40.83676699999999', '-73.918516')
('Sheridan Ave and Marcy Pl', '$1250', '1', '1', 'http://www.rent.com/new-york/new-york-houses/sheridan-ave-and-marcy-pl-4-lv25964592', '40.8371273', '-73.91370859999999')
('1427 Wythe Place', '$1250', '1', '1', 'http://www.rent.com/new-york/new-york-houses/1427-wythe-place-4-lv25964714', '40.8401172', '-73.9146257')
('2211 Cen

In [7]:
cursor.close()
db.close()